<a href="https://colab.research.google.com/github/ezequielfernandez/pln-practicas/blob/main/Ejercicio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

In [4]:
!unzip -q preprocessed_data.csv.zip 

In [5]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
#df = pd.read_csv('stephenking.txt', sep='    ', header=None)
df = pd.read_csv('preprocessed_data.csv')

df.head()

,title,text,wikipedia_title,publication_date,first_published_in,classification,notes,normalized_date
0,A DESCENT INTO THE MAELSTRÖM,"The ways of God in Nature, as in Providence, a...",A Descent into the Maelström,May 1841,Graham's Magazine,Adventure,?,May 1841
1,A PREDICAMENT,"What chance, good lady, hath bereft you thus? ...",A Predicament,November 1838,Baltimore American Museum,Parody,"Companion to ""How to Write a Blackwood Article...",November 1838
2,A TALE OF JERUSALEM,Intensos rigidarn in frontern ascendere canos ...,A Tale of Jerusalem,"June 9, 1832",Philadelphia Saturday Courier,Humor,?,June 1832
3,A TALE OF THE RAGGED MOUNTAINS,"DURING the fall of the year 1827, while residi...",A Tale of the Ragged Mountains,April 1844,Godey's Lady's Book,"Adventure,Science_fiction",?,April 1844
4,BERENICE,"Dicebant mihi sodales, si sepulchrum amicae vi...",Berenice,March 1835,Southern Literary Messenger,Horror,?,March 1835


In [6]:
df['text']

0     The ways of God in Nature, as in Providence, a...
1     What chance, good lady, hath bereft you thus? ...
2     Intensos rigidarn in frontern ascendere canos ...
3     DURING the fall of the year 1827, while residi...
4     Dicebant mihi sodales, si sepulchrum amicae vi...
                            ...                        
65    [Astounding News by Express, via Norfolk!—The ...
66    A SEQUEL TO “THE MURDERS IN THE RUE MORGUE.”  ...
67    IN speaking of the Poetic Principle, I have no...
68    BY late accounts from Rotterdam, that city see...
69    AS it is well known that the ‘wise men’ came ‘...
Name: text, Length: 70, dtype: object

In [7]:
df = pd.DataFrame(df['text'])

In [8]:
df

,text
0,"The ways of God in Nature, as in Providence, a..."
1,"What chance, good lady, hath bereft you thus? ..."
2,Intensos rigidarn in frontern ascendere canos ...
3,"DURING the fall of the year 1827, while residi..."
4,"Dicebant mihi sodales, si sepulchrum amicae vi..."
...,...
65,"[Astounding News by Express, via Norfolk!—The ..."
66,A SEQUEL TO “THE MURDERS IN THE RUE MORGUE.” ...
67,"IN speaking of the Poetic Principle, I have no..."
68,"BY late accounts from Rotterdam, that city see..."


In [9]:
print("Cantidad de documentos:", df.shape[0])


Cantidad de documentos: 70


In [10]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [11]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [12]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [13]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [14]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 70


In [15]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.key_to_index))

Cantidad de words distintas en el corpus: 5982


In [16]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 2685993.25
Loss after epoch 1: 1884423.25
Loss after epoch 2: 1786430.0
Loss after epoch 3: 1772189.0
Loss after epoch 4: 1705155.5
Loss after epoch 5: 1681289.0
Loss after epoch 6: 1664517.0
Loss after epoch 7: 1646532.0
Loss after epoch 8: 1630002.0
Loss after epoch 9: 1586179.0
Loss after epoch 10: 1567604.0
Loss after epoch 11: 1552048.0
Loss after epoch 12: 1538326.0
Loss after epoch 13: 1526898.0
Loss after epoch 14: 1519398.0
Loss after epoch 15: 1511898.0
Loss after epoch 16: 1501986.0
Loss after epoch 17: 1499278.0
Loss after epoch 18: 1495050.0
Loss after epoch 19: 1493040.0


(4178301, 6727900)

In [18]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["talking"], topn=10)

[('won’t', 0.7201682925224304),
 ('persuaded', 0.7091930508613586),
 ('“if', 0.7080977559089661),
 ('pray', 0.7053864598274231),
 ('epicurus', 0.7041090726852417),
 ('rascal', 0.6981984376907349),
 ('doomed', 0.6953663229942322),
 ('madmen', 0.6947801113128662),
 ('“und', 0.694206953048706),
 ('talk', 0.6924968361854553)]

In [19]:

from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [20]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=10
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

# Sección nueva